In [3]:
import sys
import os
import tempfile
from autogen import ConversableAgent
from autogen.coding import DockerCommandLineCodeExecutor
from pathlib import Path
from llm_config import llm_config

In [9]:
llm_config

{'config_list': [{'model': 'llama3.2',
   'api_key': 'ollama    # just a placeholder',
   'base_url': 'http://localhost:11434/v1',
   'api_type': 'openai',
   'price': [0, 0]}]}

In [2]:
work_dir = Path("coding")
work_dir.mkdir(exist_ok=True)

NameError: name 'Path' is not defined

In [14]:
# creating temporrary directory to store code files
# temp_dir = tempfile.TemporaryDirectory()

# create local command line code executor
executor = DockerCommandLineCodeExecutor(
    image = "python:3.12-slim",       # we will execute our code using this already available docker image, we can reuse this image
    timeout = 10,   # timeout for each code execution in secs
    work_dir=work_dir,    # use the temp directory to store code files
    # python_executable=sys.executable    
)

# when we runthis cell the python slim image will be created in our docker


# creating an agent with code executor configuration that uses docker
code_executor_agent_docker = ConversableAgent(
    "code_executor_agent_docker",
    llm_config=False,    # we have turned off LLM for this agent
    # max_consecutive_auto_reply = 6,
    code_execution_config={"executor": executor},     # use the docker command line code executor
    human_input_mode="NEVER",  
    is_termination_msg=lambda msg: "exit_code: 0" in msg.get("content", ""),    
)

In [15]:
code_writer_system_message = """You are a helpful AI assistant.
Solve tasks using your coding and language skills.
In the following cases, suggest python code (in a python coding block) or shell script (in a sh coding block) for the user to execute.
1. When you need to collect info, use the code to output the info you need, for example, browse or search the web, download/read a file, print the content of a webpage or a file, get the current date/time, check the operating system. After sufficient info is printed and the task is ready to be solved based on your language skill, you can solve the task by yourself.
2. When you need to perform some task with code, use the code to perform the task and output the result. Finish the task smartly.
Solve the task step by step if you need to. If a plan is not provided, explain your plan first. Be clear which step uses code, and which step uses your language skill.
When using code, you must indicate the script type in the code block. The user cannot provide any other feedback or perform any other action beyond executing the code you suggest. The user can't modify your code. So do not suggest incomplete code which requires users to modify. Don't use a code block if it's not intended to be executed by the user.
If you want the user to save the code in a file before executing it, put # filename: <filename> inside the code block as the first line. Don't include multiple code blocks in one response. Do not ask users to copy and paste the result. Instead, use 'print' function for the output when relevant. Check the execution result returned by the user.
If the result indicates there is an error, fix the error and output the code again. Suggest the full code instead of partial code or code changes. If the error can't be fixed or if the task is not solved even after the code is executed successfully, analyze the problem, revisit your assumption, collect additional info you need, and think of a different approach to try.
When you find an answer, verify the answer carefully. Include verifiable evidence in your response if possible.
Do NOT write TERMINATE anywhere.

"""

code_writer_agent = ConversableAgent(
    "code_writer_agent",
    system_message=code_writer_system_message,
    llm_config=llm_config,
    code_execution_config=False,  # Turn off code execution for this agent.
     human_input_mode="NEVER",
    # max_consecutive_auto_reply=5,
)

In [ ]:
if __name__ == "__main__":
    chat_result = code_executor_agent_docker.initiate_chat(
        code_writer_agent,
        message="Write Python code to calculate and print the 12th Fibonacci number (zero indexed).",
    )

code_executor_agent_docker (to code_writer_agent):

Write Python code to calculate and print the 12th Fibonacci number.

--------------------------------------------------------------------------------
code_writer_agent (to code_executor_agent_docker):

```python
# filename: fibonacci.py

def fibonacci(n):
    """Calculate the nth Fibonacci number."""
    if n <= 0:
        return "Input should be a positive integer."
    elif n == 1:
        return 0
    elif n == 2:
        return 1
    else:
        fib_sequence = [0, 1]
        while len(fib_sequence) < n:
            fib_sequence.append(fib_sequence[-1] + fib_sequence[-2])
        return fib_sequence[-1]

def main():
    """Print the 12th Fibonacci number."""
    print("The 12th Fibonacci number is: ", fibonacci(12))

if __name__ == "__main__":
    main()
```

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING CODE BLOCK (inferred language is python)...
code_executor_agent_docker (

In [8]:
from pprint import pprint
pprint(chat_result.chat_history)

[{'content': 'Write Python code to calculate and print the 12th Fibonacci '
             'number.',
  'name': 'code_executor_agent_docker',
  'role': 'assistant'},
 {'content': '```python\n'
             '# filename: fibonacci.py\n'
             '\n'
             'def fibonacci(n):\n'
             '    """Calculate the nth Fibonacci number."""\n'
             '    if n <= 0:\n'
             '        return "Input should be a positive integer."\n'
             '    elif n == 1:\n'
             '        return 0\n'
             '    elif n == 2:\n'
             '        return 1\n'
             '    else:\n'
             '        fib_sequence = [0, 1]\n'
             '        while len(fib_sequence) < n:\n'
             '            fib_sequence.append(fib_sequence[-1] + '
             'fib_sequence[-2])\n'
             '        return fib_sequence[-1]\n'
             '\n'
             'def main():\n'
             '    """Print the 12th Fibonacci number."""\n'
             '    print("Th

In [ ]:
if __name__ == "__main__":
    chat_result = code_executor_agent_docker.initiate_chat(
        code_writer_agent,
        message="Write Python code to line chart. Assume the data, and save the plot to a file named 'line_chart.png' Install the required dependencies .",
    )

code_executor_agent_docker (to code_writer_agent):

Write Python code to line chart. Assume the data, and save the plot to a file named 'line_chart.png' Install the required dependencies .

--------------------------------------------------------------------------------
code_writer_agent (to code_executor_agent_docker):

```sh
# filename: line_chart_generator.py

from PIL import Image, ImageDraw
import io
import matplotlib.pyplot as plt
import numpy as np

# Data for our line chart
years = [2018, 2019, 2020, 2021, 2022]
sales = [200, 220, 240, 260, 280]

# Create a new figure, and a set of subplots
fig, ax = plt.subplots()

# Plot our data on the axes
ax.plot(years, sales)

# Set some labels for the plot
ax.set_xlabel('Year')
ax.set_ylabel('Sales')

# Show the plot
plt.show(block=False)
plt.ion(True)
plt.gcf().autofmt_xdate()

# Save the plot to a file
buf = io.BytesIO()
fig.canvas.print_to_file(buf)

img = Image.open(buf).crop((0,70,150,180)).convert('RGB')

img.save('line_chart.png')